# 2025 COMP90042 Project
*Make sure you change the file name with your group id.*

# Readme
*If there is something to be noted for the marker, please mention here.*

*If you are planning to implement a program with Object Oriented Programming style, please put those the bottom of this ipynb file*

# 1.DataSet Processing
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)

In [1]:
# Monte the files from MyDrive
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# install packages for hugging face
!pip install sentence-transformers faiss-cpu tqdm nltk datasets lemminflect spacy



^C


   ---------------------------------------- 0.0/15.0 MB ? eta -:--:--
   ---- ----------------------------------- 1.8/15.0 MB 9.2 MB/s eta 0:00:02
   ------------- -------------------------- 5.0/15.0 MB 13.1 MB/s eta 0:00:01
   --------------------- ------------------ 8.1/15.0 MB 14.0 MB/s eta 0:00:01
   --------------------------- ------------ 10.5/15.0 MB 13.4 MB/s eta 0:00:01
   -------------------------------- ------- 12.3/15.0 MB 12.3 MB/s eta 0:00:01
   ------------------------------------- -- 14.2/15.0 MB 11.9 MB/s eta 0:00:01
   ---------------------------------------- 15.0/15.0 MB 11.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/769.7 kB ? eta -:--:--
   ---------------------------------------- 769.7/769.7 kB 5.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/212.5 MB ? eta -:--:--
    --------------------------------------- 3.1/212.5 MB 16.8 MB/s eta 0:00:13
   - -------------------------------------- 6.6/212.5 MB 16.8 MB/s eta 0:00:13
 

In [31]:
# Import all the packages
import numpy as np
import json
from collections import Counter
from sentence_transformers import SentenceTransformer
import pickle
import faiss
from tqdm import tqdm
import nltk
import re
from datasets import Dataset
from nltk.tokenize import sent_tokenize
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [9]:
# Analyse train-claims and evidences

train_claims_path = './data/train-claims.json'
dev_claims_path = './data/dev-claims.json'
evidence_path = './data/evidence.json'

def analyze_data(train_claims_path, dev_claims_path, evidence_path):
  # Load the datasets
  with open(train_claims_path, 'r') as f:
    train_claims = json.load(f)
  with open(dev_claims_path, 'r') as f:
    dev_claims = json.load(f)
  with open(evidence_path, 'r') as f:
    evidences = json.load(f)

  #  Analyze train-claims.json
  print("=== Train Claims Analysis ===")
  print(f"Total train claims: {len(train_claims)}")
  # Claim length (in characters and tokens) & evidence count
  claim_lengths_char = []
  claim_lengths_token = []
  evidence_counts = []
  claim_labels = Counter()

  for claim in train_claims.values():
    claim_text = claim["claim_text"]
    claim_lengths_char.append(len(claim_text))
    claim_lengths_token.append(len(claim_text.split()))
    evidence_counts.append(len(claim.get("evidences", [])))
    claim_labels[claim["claim_label"]] += 1

  print(f"Max claim length (characters): {max(claim_lengths_char)}")
  print(f"Min claim length (characters): {min(claim_lengths_char)}")
  print(f"Mean claim length (characters): {np.mean(claim_lengths_char):.1f}")
  print(f"Max claim length (tokens): {max(claim_lengths_token)}")
  print(f"Min claim length (tokens): {min(claim_lengths_token)}")
  print(f"Mean claim length (tokens): {np.mean(claim_lengths_token):.1f}")

  print(f"Max evidence count: {max(evidence_counts)}")
  print(f"Min evidence count: {min(evidence_counts)}")
  print(f"Mean evidence count: {np.mean(evidence_counts):.2f}")

  print("Claim label distribution:")
  for label, count in claim_labels.items():
    print(f"  {label}: {count} ({count / len(train_claims):.2%})")

  # Analyze dev-claims.json
  print("=== Dev Claims Analysis ===")
  print(f"Total dev claims: {len(dev_claims)}")
  # Claim length (in characters and tokens) & evidence count
  claim_lengths_char = []
  claim_lengths_token = []
  evidence_counts = []
  claim_labels = Counter()

  for claim in dev_claims.values():
    claim_text = claim["claim_text"]
    claim_lengths_char.append(len(claim_text))
    claim_lengths_token.append(len(claim_text.split()))
    evidence_counts.append(len(claim.get("evidences", [])))
    claim_labels[claim["claim_label"]] += 1

  print(f"Max claim length (characters): {max(claim_lengths_char)}")
  print(f"Min claim length (characters): {min(claim_lengths_char)}")
  print(f"Mean claim length (characters): {np.mean(claim_lengths_char):.1f}")
  print(f"Max claim length (tokens): {max(claim_lengths_token)}")
  print(f"Min claim length (tokens): {min(claim_lengths_token)}")
  print(f"Mean claim length (tokens): {np.mean(claim_lengths_token):.1f}")

  print(f"Max evidence count: {max(evidence_counts)}")
  print(f"Min evidence count: {min(evidence_counts)}")
  print(f"Mean evidence count: {np.mean(evidence_counts):.2f}")

  print("Claim label distribution:")
  for label, count in claim_labels.items():
    print(f"  {label}: {count} ({count / len(dev_claims):.2%})")
  # Analyze evidences.json
  print("\n=== Evidence Analysis ===")
  print(f"Total evidences: {len(evidences)}")
  evidence_lengths_char = [len(text) for text in evidences.values()]
  evidence_lengths_token = [len(text.split()) for text in evidences.values()]

  print(f"Max evidence length (characters): {max(evidence_lengths_char)}")
  print(f"Min evidence length (characters): {min(evidence_lengths_char)}")
  print(f"Mean evidence length (characters): {np.mean(evidence_lengths_char):.1f}")
  print(f"Max evidence length (tokens): {max(evidence_lengths_token)}")
  print(f"Min evidence length (tokens): {min(evidence_lengths_token)}")
  print(f"Mean evidence length (tokens): {np.mean(evidence_lengths_token):.1f}")

# Example usage:
analyze_data(train_claims_path, dev_claims_path, evidence_path)


=== Train Claims Analysis ===
Total train claims: 1228
Max claim length (characters): 332
Min claim length (characters): 26
Mean claim length (characters): 123.0
Max claim length (tokens): 67
Min claim length (tokens): 4
Mean claim length (tokens): 20.1
Max evidence count: 5
Min evidence count: 1
Mean evidence count: 3.36
Claim label distribution:
  DISPUTED: 124 (10.10%)
  REFUTES: 199 (16.21%)
  SUPPORTS: 519 (42.26%)
  NOT_ENOUGH_INFO: 386 (31.43%)
=== Dev Claims Analysis ===
Total dev claims: 154
Max claim length (characters): 406
Min claim length (characters): 31
Mean claim length (characters): 128.0
Max claim length (tokens): 65
Min claim length (tokens): 4
Mean claim length (tokens): 21.1
Max evidence count: 5
Min evidence count: 1
Mean evidence count: 3.19
Claim label distribution:
  SUPPORTS: 68 (44.16%)
  NOT_ENOUGH_INFO: 41 (26.62%)
  REFUTES: 27 (17.53%)
  DISPUTED: 18 (11.69%)

=== Evidence Analysis ===
Total evidences: 1208827
Max evidence length (characters): 3148
Min ev

In [32]:
train_claims_path = './data/train-claims.json'
dev_claims_path = './data/dev-claims.json'
evidence_path = './data/evidence.json'

In [33]:
from sentence_transformers import CrossEncoder, InputExample
from torch.utils.data import DataLoader
import json
import random

train = False
dev_train_included = False

if train:
    with open(train_claims_path, 'r') as f:
        train_claims = json.load(f)
    with open(evidence_path, 'r') as f:
        evidence_dict = json.load(f)
    if dev_train_included:
        with open(dev_claims_path, 'r') as f:
            dev_claims = json.load(f)

    # ✅ 构造正负样本
    train_samples = []

    def generate_samples(claims_data):
        samples = []
        for claim in claims_data.values():
            claim_text = claim["claim_text"]
            evidence_ids = claim.get("evidences", [])
            pos_evidence_texts = [evidence_dict[eid] for eid in evidence_ids if eid in evidence_dict]

            # 正样本
            for ev in pos_evidence_texts:
                samples.append(InputExample(texts=[claim_text, ev], label=1.0))

            # 负样本
            neg_pool = [e for eid, e in evidence_dict.items() if eid not in evidence_ids]
            for _ in range(len(pos_evidence_texts)):
                neg_ev = random.choice(neg_pool)
                samples.append(InputExample(texts=[claim_text, neg_ev], label=0.0))

        return samples

    train_samples.extend(generate_samples(train_claims))
    if dev_train_included:
        train_samples.extend(generate_samples(dev_claims))

    print(f"✅ Total training samples: {len(train_samples)}")

    # ✅ 构建 DataLoader（InputExample 是合法格式）
    train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=16)

    # ✅ 加载 MS MARCO CrossEncoder
    model = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2", num_labels=1)

    # ✅ 训练模型
    model.fit(
        train_dataloader=train_dataloader,
        epochs=5,
        warmup_steps=100,
        show_progress_bar=True
    )

    # ✅ 保存模型
    if (dev_train_included):
        model.save('./model/final_finetuned_msmarco_reranker')
        print("✅ Finetuned model saved.")
    else:
        model.save('./model/my_finetuned_msmarco_reranker')
        print("✅ Finetuned model saved.")


In [34]:
if (dev_train_included):
    model = SentenceTransformer('./model/final_finetuned_minilm_retriever')
    reranker =  CrossEncoder('./model/final_finetuned_msmarco_reranker')
else:
    model = SentenceTransformer('./model/my_finetuned_minilm_retriever')
    reranker =  CrossEncoder('./model/my_finetuned_msmarco_reranker')

In [26]:
import nltk
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter
from lemminflect import getAllInflections
nlp = spacy.load("en_core_web_sm")
with open(train_claims_path, 'r') as f:
    train_claims = json.load(f)

with open(evidence_path, 'r') as f:
    evidence_dict = json.load(f)

# === 从 claim_text 中提取所有名词并统计词频 ===
all_nouns = []
for claim_obj in train_claims.values():
    doc = nlp(claim_obj["claim_text"])
    nouns = [token.lemma_.lower() for token in doc if token.pos_ == "NOUN"]
    all_nouns.extend(nouns)

# === 选出 top 100 高频名词作为关键词 ===
top_keywords = set(word for word, _ in Counter(all_nouns).most_common(100))

all_forms = set()
for lemma in top_keywords:
  all_forms.add(lemma)
  # 拿到所有可能的名词形式
  infl_map = getAllInflections(lemma, upos="NOUN")
  # infl_map 是个 dict：{ 'NNS': ['cats'], 'NNPS': ['children'], ... }
  for forms in infl_map.values():
      all_forms.update(forms)


In [35]:
def contains_climate_keywords(text: str, all_forms: set) -> bool:
    # 直接把文本小写化、拆成“词”后查集合
    words = re.findall(r"\b[a-z']+\b", text.lower())
    return any(word in all_forms for word in words)


def is_english(text: str, threshold: float = 0.5) -> bool:
  # 清理文本，仅保留字母和空格
  text = re.sub(r'[^a-zA-Z\s]', '', text)
  if len(text) == 0:  # 如果文本清理后为空，返回False
    return False
  # 计算英文字符占比
  english_char_count = sum(1 for char in text if char.isalpha())
  return (english_char_count / len(text)) >= threshold

def clean_and_split(eid, text):
  result_ids = []
  result_texts = []
  sentences = sent_tokenize(text)
  for i, sent in enumerate(sentences):
    sent = sent.lower()
    sent = re.sub(r'[^a-z0-9\s.,!?]', '', sent)  # 去除标点
    sent = re.sub(r'\s+', ' ', sent).strip()
    if len(sent.split()) >= 5:  # 可选：过滤太短的文本
      result_ids.append(f"{eid}_s{i}")  # 用原始的 eID 加上句子索引
      result_texts.append(sent)
  return result_ids, result_texts



In [36]:
encode = False # Variable to control whether encode the evidence
# Load the sentence-BERT (all-MiniLM-L6-v2)
if (dev_train_included):
  word_embedding_path = './word_embedding/final_evidence_embeddings.npy'
  word_embedding_meta_path = "./word_embedding/final_evidence_meta.pkl"
else: 
  word_embedding_path = './word_embedding/evidence_embeddings.npy'
  word_embedding_meta_path = "./word_embedding/evidence_meta.pkl"

if (encode):
  with open(evidence_path, 'r') as f:
    evidence_dict = json.load(f)
  # ——— 1. 批量剔除非英文 —— #
  eids  = list(evidence_dict.keys())
  texts = list(evidence_dict.values())
  english_pairs = [
    (eid, txt)
    for eid, txt in zip(eids, texts)
    if is_english(txt)
  ]
  print(f"Step1: English keep {len(english_pairs)}/{len(texts)}")

  # ——— 2. 批量剔除非 climate-related —— #
  climate_pairs = [
    (eid, txt)
    for eid, txt in english_pairs
    if contains_climate_keywords(txt, all_forms)
  ]
  print(f"Step2: Climate-related keep {len(climate_pairs)}/{len(english_pairs)}")


  # 清洗和分句后的结果
  cleaned_evidence_ids = []
  cleaned_evidence_texts = []
  original_evidence_ids = []  # 记录每个分句所属的原始 evidence_id

  # 遍历 evidence 数据并进行清洗和分句
  for eid, text in climate_pairs:
    cleaned_ids, cleaned_texts = clean_and_split(eid, text)
    cleaned_evidence_ids.extend(cleaned_ids)
    cleaned_evidence_texts.extend(cleaned_texts)
    original_evidence_ids.extend([eid] * len(cleaned_ids))  # 每个分句都记录原始的 eID

  # 使用 Sentence-BERT 对清洗后的分句进行编码
  evidence_embeddings = model.encode(
    cleaned_evidence_texts,
    convert_to_numpy=True,
    normalize_embeddings=True,
    show_progress_bar=True
  )

  # 保存编码后的嵌入（embeddings）
  np.save(word_embedding_path, evidence_embeddings)

  # 保存分句后的 evidence_ids 和 texts 以及对应的原始 evidence_id 映射
  with open(word_embedding_meta_path, "wb") as f:
      pickle.dump((cleaned_evidence_ids, cleaned_evidence_texts, original_evidence_ids), f)


# 2. Model Implementation
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)

# 3.Testing and Evaluation
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)

In [40]:
# Load numpy embeddings
evidence_embeddings = np.load(word_embedding_path)

# Load evidence_ids, evidence_texts, and original_evidence_ids
with open(word_embedding_meta_path, "rb") as f:
  evidence_ids, evidence_texts, original_evidence_ids = pickle.load(f)


dimension = evidence_embeddings.shape[1]
index = faiss.IndexFlatIP(dimension)  # Inner Product = Cosine similarity if normalized
index.add(evidence_embeddings)


def clean_claim(claim: str) -> str:
  # 小写化
  claim = claim.lower()
  # 去除标点符号
  claim = re.sub(r'[^a-z0-9\s]', '', claim)
  # 去除多余空格
  claim = re.sub(r'\s+', ' ', claim).strip()
  # 如果需要，你也可以添加去除停用词的步骤
  # claim = " ".join([word for word in claim.split() if word not in stopwords])
  return claim

# Create mapping
evidence_dict = dict(zip(evidence_ids, evidence_texts))

with open(evidence_path, 'r') as f:
    original_evidence_dict = json.load(f)

def retrieve_evidence(claim_id, claim_data, retrieval=100, top_k=5):
    claim_text = claim_data["claim_text"]
    cleaned_claim = clean_claim(claim_text)

    # Step 1: 粗检索（model + FAISS）
    claim_embedding = model.encode([cleaned_claim], convert_to_numpy=True, normalize_embeddings=True)
    scores, indices = index.search(claim_embedding, retrieval * 3)

    # Step 2: 去重候选
    seen_original_ids = set()
    candidates = []
    for i in indices[0]:
        eid = evidence_ids[i]
        text = evidence_dict[eid]
        original_id = original_evidence_ids[i]

        if original_id not in seen_original_ids:
            candidates.append((original_id, eid, text))
            seen_original_ids.add(original_id)

        if len(candidates) >= retrieval:
            break

    # Step 3: 精排序（CrossEncoder）
    pairs = [(claim_text, original_evidence_dict[orig_id]) for (orig_id, _, _) in candidates]  # ✅
    similarity_scores = reranker.predict(pairs)

    reranked = sorted(zip(candidates, similarity_scores), key=lambda x: x[1], reverse=True)

    # ✅ 只返回原始 document-level evidence ID
    top_k_original_ids = [orig_id for (orig_id, _, _), _ in reranked[:top_k]]

    # ✅ 构造最终结果 dict
    result = {
        "claim_text": claim_text,
        "evidences": top_k_original_ids
    }

    return result


In [55]:
from tqdm import tqdm
import numpy as np
import json

# 加载 dev claims
with open(dev_claims_path, 'r') as f:
    dev_claims = json.load(f)

claim_ids = list(dev_claims.keys())
top_k = 4

recalls = []
precisions = []
f1s = []

for cid in tqdm(claim_ids, desc="Evaluating"):
    truth = set(dev_claims[cid]["evidences"])
    
    # ✅ 新版 retrieve_evidence 返回 dict
    retrieved_info = retrieve_evidence(cid, dev_claims[cid], retrieval=100, top_k=top_k)
    retrieved = set(retrieved_info["evidences"])

    hit = len(truth & retrieved)

    # Recall
    recall = hit / len(truth) if len(truth) > 0 else 0
    recalls.append(recall)

    # Precision
    precision = hit / top_k if top_k > 0 else 0
    precisions.append(precision)

    # F1
    if precision + recall > 0:
        f1 = 2 * precision * recall / (precision + recall)
    else:
        f1 = 0
    f1s.append(f1)

# 输出平均指标
print(f"\n📊 Average Recall@{top_k}:    {np.mean(recalls):.2%}")
print(f"📊 Average Precision@{top_k}: {np.mean(precisions):.2%}")
print(f"📊 Average F1@{top_k}:        {np.mean(f1s):.2%}")


Evaluating: 100%|██████████| 154/154 [00:17<00:00,  9.06it/s]


📊 Average Recall@4:    26.36%
📊 Average Precision@4: 18.02%
📊 Average F1@4:        19.90%


In [52]:
import json
import re
import numpy as np
from tqdm import tqdm

# 加载 test claims 数据
with open('./data/test-claims-unlabelled.json', 'r') as f:
    test_claims = json.load(f)

# 清洗函数
def clean_claim(claim: str) -> str:
    claim = claim.lower()
    claim = re.sub(r'[^a-z0-9\s]', '', claim)
    claim = re.sub(r'\s+', ' ', claim).strip()
    return claim

# ✅ 执行检索并保存
test_predictions = {}

for claim_id, claim_data in tqdm(test_claims.items(), desc="Retrieving Evidence"):
    result = retrieve_evidence(claim_id, claim_data, retrieval=100, top_k=4)

    # 去掉 claim_label（因为 test 中无 label）
    if "claim_label" in result:
        del result["claim_label"]

    test_predictions[claim_id] = result

# ✅ 保存预测结果
with open("test-claims-predictions.json", "w") as f:
    json.dump(test_predictions, f, indent=2)

print("✅ Retrieval with reranker completed and saved to test-claims-predictions.json")


Retrieving Evidence: 100%|██████████| 153/153 [00:16<00:00,  9.11it/s]

✅ Retrieval with reranker completed and saved to test-claims-predictions.json


In [53]:
import json
import numpy as np

# === 文件路径（请替换为你自己的路径） ===
prediction_file = 'test-claims-predictions.json'
groundtruth_file = 'groundtruth_output.json'

# === 加载数据 ===
with open(prediction_file, 'r') as f:
    predictions = json.load(f)

with open(groundtruth_file, 'r') as f:
    groundtruth = json.load(f)

# === 指标累加器 ===
f_scores = []
precisions = []
recalls = []

# === 遍历每个 claim ===
for claim_id, gt_entry in groundtruth.items():
    if claim_id not in predictions:
        continue

    pred_entry = predictions[claim_id]

    # 提取并转换为集合（防止重复）
    gold = set(gt_entry.get("evidences", []))
    raw_pred = pred_entry.get("evidences", [])

    # 若是 list of lists，做扁平化；否则直接处理
    if raw_pred and isinstance(raw_pred[0], list):
        pred = set(eid for group in raw_pred for eid in group)
    else:
        pred = set(raw_pred)

    if not pred:
        precisions.append(0.0)
        recalls.append(0.0)
        f_scores.append(0.0)
        continue

    # 计算指标
    correct = gold & pred
    precision = len(correct) / len(pred)
    recall = len(correct) / len(gold) if gold else 0.0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) else 0.0

    precisions.append(precision)
    recalls.append(recall)
    f_scores.append(f1)

# === 输出平均指标 ===
print("\n📊 Evidence Retrieval Evaluation")
print(f" - Mean Precision: {np.mean(precisions):.4f}")
print(f" - Mean Recall   : {np.mean(recalls):.4f}")
print(f" - Mean F1 Score : {np.mean(f_scores):.4f}")



📊 Evidence Retrieval Evaluation
 - Mean Precision: 0.1875
 - Mean Recall   : 0.2816
 - Mean F1 Score : 0.2087


## Object Oriented Programming codes here

*You can use multiple code snippets. Just add more if needed*